In [1]:
import os
import io
import sys
from dotenv import load_dotenv
import google.generativeai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [3]:
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists")
else:
    print("Google API Key not set")

Google API Key exists


In [4]:
google.generativeai.configure()

In [5]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Windows system. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [6]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [8]:
def write_output(cpp):
    code = cpp.replace("```cpp", "").replace("```", "")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [9]:
def optimize_google(python):
    reply = ""

    gemini = google.generativeai.GenerativeModel(
        model_name="gemini-1.5-pro",
        system_instruction=system_message
    )

    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )

    for chunk in response:
        if chunk.text:
            reply += chunk.text
            print(chunk.text, end="", flush=True)

    write_output(reply)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 13.346197 seconds


In [12]:
optimize_google(pi)

```cpp
#include <iostream>
#include <iomanip>
 <chrono>

int main() {
00000000;long iterations = 1
    int param1 = 4;
    int param2 = 1;

time = std::chrono::high_resolution_clock::now();

    double result = 1.0;
 1; i <= iterations; ++i) {
        long long j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    result *= 4.0;

    auto end_time = std::chrono::high_resolution_clock::now();
duration<double> elapsed_time = end_time - start_time;


    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
 "Execution Time: " << elapsed_time.count() << " seconds" << std::endl;

    return 0;
}

```

In [13]:
def stream_google(python):
    # Initialize empty reply string
    reply = ""
    
    # The API for Gemini has a slightly different structure
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-pro',
        system_instruction=system_message
    )
    
    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )
    
    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```cpp\n','').replace('```','')


In [14]:
def optimize(python, model):
    result=stream_google(python)
    for stream_so_far in result:
        yield stream_so_far    

In [16]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["Google"], label="Select model", value="Google")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [17]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [18]:
def execute_cpp(code):
    write_output(code)
    try:
        compile_cmd = ["g++", "-o", "optimized", "optimized.cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [19]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [20]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["Google"], label="Select model", value="Google")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.Textbox(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\seoli\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\seoli\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\seoli\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\seoli\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\seoli\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai